In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict

import json
import pandas as pd

from scipy.stats import ttest_ind
from numpy import random

DEFAULT_RANDOM_SEED = 774
random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)

In [ ]:
category = "min_tpm_5"

In [ ]:
extra_data_headers = pd.read_csv('../data/extra_data.tsv', delimiter="\t", nrows=0).columns
data = pd.read_csv(f"../preprocessed/{category}/genes.csv", delimiter=",", decimal='.')

subtypes = set(data["subtype"])
genes = set(data.columns) - set(extra_data_headers) - set(["prognostic"])

In [ ]:
important_genes_by_sex_subtype = defaultdict(list)

for sex in ["Male", "Female"]:
  filtered_sex_dataset = data[data["sex"] == sex]
  for subtype in subtypes:
    subtype_dataset = filtered_sex_dataset[filtered_sex_dataset["subtype"] == subtype]
    not_subtype_dataset = filtered_sex_dataset[filtered_sex_dataset["subtype"] != subtype]

    for gene in genes:
      # Do populations have identical variances?
      result = ttest_ind(subtype_dataset[gene], not_subtype_dataset[gene], equal_var=False)
      if (result.pvalue <= 0.001):
        important_genes_by_sex_subtype[(sex, subtype)].append({ "gene": gene, "pvalue": result.pvalue })


In [68]:
result = defaultdict(dict)
for key in important_genes_by_sex_subtype.keys():
  result[key[1]] |= { key[0]: sorted(important_genes_by_sex_subtype[key], key=lambda x: x["pvalue"]) }

In [ ]:
open(f"../preprocessed/{category}/important_genes_pvalue.json", "w").write(json.dumps(result))

1923488